# markdown.obsidian.personal.vault

> Manage a vault in ways that are specific to `trouver`

In [ ]:
#| default_exp markdown.obsidian.personal.vault

In [ ]:
#| export
from pathlib import Path
import os
from os import PathLike

from trouver.helper.constants import ALPHABETICAL_SUBDIRECTORIES

In [ ]:
import shutil
import tempfile
from unittest import mock

from fastcore.test import *

from trouver.helper.tests import _test_directory

# Setup an `Obsidian.md` vault for using `trouver`

`trouver` currently requires `Obsidian.md` vaults to be setup in specific ways[^1]. 

[^1]: Most representatively, `trouver` curreently requires the vault to have a directories called `_references` and `_templates` in the root directory of the vault. These directories, need to have subdirectories named `A-E`, `F-J`, `K-O`, `P-T`, `U-Z`, each of which need to have further subdirectories whose names are single letter alphabets in the ranges of their own names, e.g. `A-E` has subdirectories named `A`, `B`, `C`, `D`, and `E`. 

    These directories store reference notes and template notes, see ``14_markdown.obsidian.personal.note_type``

In [ ]:
#| export
def setup_obsidian_vault_for_trouver(
        vault: PathLike, # The path to the vault to setup.
        verbose: bool = True, # If true, print messages explaining what is created
        prompt: bool = True, # If true, prompt user input for certain decisions, such as whether to create an Obsidian vault folder if `vault` does not exist.
    ):
    # TODO: use this method for vault construction for a reference
    """Setup an `Obsidian.md` vault for `trouver`.
    
    Currently, the following subdirectories are created in `vault` if they do not
    already exist:

    - `_references`, `_templates`
        - Each of these directories are then populated by subdirectories 
          `A-E`, `F-J`, `K-O`, `P-T`, `U-Z`, and those subdirectories are
          further populated by subdirectories whose names are single-letter
          alphabets in the ranges of these names, e.g. `A-E` has subdirectories
          `A`, `B`, `C`, `D`, and `E`.
    """
    _setup_trouver_vault_essentials(Path(vault), verbose, prompt)


def _setup_trouver_vault_essentials(
        vault: PathLike, # The path to the vault to setup
        verbose: bool, # If true, print messages explaining what is created
        prompt: bool, # If true, prompt user input for certain decisions, such as whether to create an Obsidian vault folder if `vault` does not exist.
    ):
    """
    """
    # TODO: setup missing subfolders in the `_references` or `_templates` subfolders even
    for name in ['_references', '_templates']:
        if os.path.exists(vault / name) and verbose:
            print(f"The vault already has a `{name}` folder.")
        elif not os.path.exists(vault / name) and prompt:
            # TODO do input things
            command = input(
                f"""The vault does not have a `{name}` folder.
                Create it and subdirectories?\n[Y/(n)]""")
        # if not os.path.exists(vault / name) and ()
        #     print('')
        if not os.path.exists(vault / name) and (not prompt or command == 'Y'):
            if verbose:
                print(f"Creating `{name}` folder in the vault.")
            _setup_alphabetical_subdirectories(vault, name)



def _setup_alphabetical_subdirectories(
        dir: PathLike, # The path to the directory in which to make the subdirectory with alphabetical subdirectories
        name: str, # The name of the subdirecctory to make in which the alphabetical subdirectories will be made
    ):
    """Create a subdirectory with name ``name`` in the directory ``dir`` and create
    alphabetical subdirectories.
    
    Assumes that ``dir`` exists, but the subdirectory with name ``name`` does not.
    """
    os.mkdir(dir / name)
    for sub_name in ALPHABETICAL_SUBDIRECTORIES:
        os.mkdir(dir / name / sub_name)
        first_letter, last_letter = ord(sub_name[0]), ord(sub_name[-1])
        for i in range(first_letter, last_letter+1):
            os.mkdir(dir / name / sub_name / chr(i))




In [ ]:
#| hide
with mock.patch('os.mkdir') as mock_mkdir:
    _setup_alphabetical_subdirectories(Path('.'), '_references')
    
    # 1. Verify all top-level alphabetical folders are created
    for subdir in ALPHABETICAL_SUBDIRECTORIES:
        expected_call = mock.call(Path('_references') / subdir)
        assert expected_call in mock_mkdir.call_args_list

    # 2. Verify a few specific nested folders exist (spot check)
    # Since you know "A-E" contains "A", you can still keep specific spot checks 
    # if you want, or make this loop dynamic too.
    
    assert mock.call(Path('_references') / 'A-E') in mock_mkdir.call_args_list
    assert mock.call(Path('_references') / 'U-Z') in mock_mkdir.call_args_list
    assert mock.call(Path('_references') / 'A-E' / 'A') in mock_mkdir.call_args_list
    assert mock.call(Path('_references') / 'K-O' / 'L') in mock_mkdir.call_args_list

In [ ]:
#| hide
with (mock.patch('__main__.input', return_value='Y') as mock_input,
      mock.patch('__main__._setup_alphabetical_subdirectories') as mock_setup_alphabetical_subdirectories):
    mock_vault = Path('.')
    _setup_trouver_vault_essentials(vault=mock_vault, verbose=True, prompt=True)

    test_eq(
        mock_setup_alphabetical_subdirectories.call_args_list,
        [mock.call(mock_vault, '_references'), mock.call(mock_vault, '_templates')])



Creating `_references` folder in the vault.
Creating `_templates` folder in the vault.


In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    # Setup paths
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)

    # Run the function
    setup_obsidian_vault_for_trouver(temp_vault, False, False)
    
    # 1. Verify specific "Root" folders exist
    # If these names are also constants (e.g. REFERENCES_DIR_NAME), use those!
    for root_folder in ['_references', '_templates']:
        base = temp_vault / root_folder
        assert base.exists(), f"Missing root folder: {root_folder}"

        # 2. Verify all standard alphabetical groups exist (A-E, F-J, etc.)
        for group_name in ALPHABETICAL_SUBDIRECTORIES:
            group_path = base / group_name
            assert group_path.exists(), f"Missing group folder: {group_path}"

    # 3. Spot Check: Verify sub-letter folders (A, B, C...) exist inside the groups
    # This verifies the logic "create 'A' inside 'A-E'" actually happened.
    # You don't need to check every single letter, just enough to know the logic ran.
    refs = temp_vault / '_references'
    assert (refs / 'A-E' / 'A').exists()
    assert (refs / 'A-E' / 'E').exists()
    
    # Check a different group in templates to ensure it works there too
    temps = temp_vault / '_templates'
    assert (temps / 'K-O' / 'K').exists()
    assert (temps / 'K-O' / 'O').exists()


with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir):
    # Vault already with the '_references' and '_templates' folder
    # The `_references` and `_templates` folder do not get re-made.
    temp_vault = Path(temp_dir) / 'empty_model_vault'
    shutil.copytree(_test_directory() / 'empty_model_vault', temp_vault)
    with (mock.patch('os.mkdir') as mock_mkdir):
        assert os.path.exists(Path(temp_dir) / 'empty_model_vault' / '_references')
        setup_obsidian_vault_for_trouver(temp_vault, False, False)
        assert os.path.exists(Path(temp_dir) / 'empty_model_vault' / '_references')
        mock_mkdir.assert_not_called()
